In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from __init__ import *

In [ ]:
ws, hs = read_ws_hs(
    "{}/mf".format(OUTPUT_DIRECTORY_PATH),
    SETTING["model_mode"],
    SETTING["model_data_dicts"],
)

In [ ]:
distance_function = kraft.compute_information_distance_between_2_vectors

heat_map_layout_size = 640

layout = {"height": heat_map_layout_size, "width": heat_map_layout_size}

In [ ]:
w_factor_x_factor_distances = np.asarray(
    [
        kraft.apply_function_on_slices_from_2_dataframes(
            w,
            w,
            0,
            distance_function,
            layout={"title": {"text": "W{} Distance".format(i)}, **layout},
        ).values
        for i, w in enumerate(ws)
    ]
)

In [ ]:
h_factor_x_factor_distances = np.asarray(
    [
        kraft.apply_function_on_slices_from_2_dataframes(
            h,
            h,
            1,
            distance_function,
            layout={"title": {"text": "H{} Distance".format(i)}, **layout},
        ).values
        for i, h in enumerate(hs)
    ]
)

In [ ]:
factor_x_factor_distances = np.concatenate(
    (w_factor_x_factor_distances, h_factor_x_factor_distances)
)

factor_x_factor_distance = pd.DataFrame(
    np.mean(factor_x_factor_distances, axis=0), index=hs[0].index, columns=ws[0].columns
)

kraft.plot_heat_map(
    factor_x_factor_distance.iloc[
        kraft.cluster_matrix(factor_x_factor_distance.values, 0),
        kraft.cluster_matrix(factor_x_factor_distance.values, 1),
    ],
    layout={"title": {"text": "W H Distance"}, **layout},
)

In [ ]:
gps_map_directory_path = "{}/gps_map".format(OUTPUT_DIRECTORY_PATH)

kraft.establish_path(gps_map_directory_path, "directory")

In [ ]:
rs_01 = kraft.normalize_array(
    np.concatenate(tuple((r,) * r for r in SETTING["mf_rs"])), "0-1"
)

node_factor = SETTING["mf_r_range"] ** rs_01

node_marker_size = 3 ** (1 - rs_01) * 16

In [ ]:
w_element_x_node = pd.concat(ws)

w_element_x_node *= node_factor

w_element_x_node **= SETTING["w_gps_map_node_power"]

In [ ]:
w_gps_map = kraft.GPSMap(
    w_element_x_node, factor_x_factor_distance, node_marker_size=node_marker_size
)

kraft.write_gps_map("{}/w_gps_map.pickle.gz".format(gps_map_directory_path), w_gps_map)

w_gps_map.plot(
    element_trace=SETTING["w_gps_map_element_trace"],
    html_file_path="{}/w_gps_map.html".format(gps_map_directory_path),
)

In [ ]:
h_element_x_node = pd.concat(hs, axis=1).T

h_element_x_node *= node_factor

h_element_x_node **= SETTING["h_gps_map_node_power"]

In [ ]:
h_gps_map = kraft.GPSMap(
    h_element_x_node, factor_x_factor_distance, node_marker_size=node_marker_size
)

kraft.write_gps_map("{}/h_gps_map.pickle.gz".format(gps_map_directory_path), h_gps_map)

h_gps_map.plot(
    element_trace=SETTING["h_gps_map_element_trace"],
    html_file_path="{}/h_gps_map.html".format(gps_map_directory_path),
)